# Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression

import numpy as np
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import os
import math

In [ ]:
# Mouting with google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Config path and variable

In [ ]:
# Path to data folder
data_folder = '/content/drive/MyDrive/STUDYING/HK2 2020-2021/Modern Problems in Technology/'

# Choose one of 2 versions below:
# Uncomment below line to choose scaled data version
# data_version = 'data_processed_withscale.csv'|

# Uncomment below line to choose non-scaled data version
data_version = 'data_processed_withoutscale.csv'

## Reading data version

In [ ]:
data = pd.read_csv(os.path.join(data_folder, data_version))

data


,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
0,21.0491,105.8831,0.000000,391.864014,102794.0,66.589302,11.087100,4.69022,5.980500,5.019875,4.711437,4.758125,27.722601,1828.880005,4
1,21.0491,105.8831,0.000000,558.942993,102699.0,65.773903,12.090000,3.59040,2.657062,4.092187,8.700875,7.603187,27.722601,1828.880005,4
2,21.0491,105.8831,41.310400,326.197998,102453.0,69.259697,13.018900,2.96978,5.755687,6.602938,8.837000,9.167312,27.722601,1828.880005,4
3,21.0491,105.8831,39.379204,197.524994,102181.0,78.663696,15.516100,2.77573,8.956062,10.830875,13.301438,13.640625,27.722601,1828.880005,4
4,21.0491,105.8831,39.901879,287.825012,102072.0,75.866699,17.658199,3.12869,12.547000,12.891687,15.638813,12.034750,27.722601,1828.880005,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,18.995833,454.903015,102068.0,43.484299,16.768499,3.66034,7.437375,5.100375,3.930875,6.508750,442.738007,4993.080078,5
5456,21.0197,105.8147,23.241667,400.079010,101755.0,49.469700,19.499901,2.41066,13.452063,12.091188,5.180625,9.206437,442.738007,4993.080078,5
5457,21.0197,105.8147,16.816667,326.407013,101510.0,66.391800,22.555500,1.45771,17.722375,16.551500,13.518500,18.223813,442.738007,4993.080078,5
5458,21.0197,105.8147,21.595833,468.182007,101796.0,64.951401,23.062800,4.10253,16.676687,14.273500,16.221062,18.177937,442.738007,4993.080078,5


In [ ]:
# data = data[data.cluster != 2]
# Unique cluster value
data['cluster'].unique()

array([4, 1, 2, 3, 0, 5])

## Remove Invalid NO2 values

In [ ]:
# Run this if you want to remove invalid no2 values

data = data[(data.NO2 > 0) & (data.NO2 < 100)]
data[data.NO2 > 100]

,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster


## Drop useless columns

In [ ]:
def drop_cols(df):
  drops = ['dpt_06', 'dpt_12']
  for col in drops:
    if col in df:
      df.drop(col, axis='columns', inplace=True)

# drop_cols(data)

## Convert to numpy for feeding model

In [ ]:
X = data.drop('NO2', axis = 'columns')
X_arr = X.to_numpy()

y = data['NO2']
y_arr = y.to_numpy()

print(X.shape, y.shape)

(4679, 14) (4679,)


In [ ]:
X

,lat,lon,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
2,21.0491,105.8831,326.197998,102453.0,69.259697,13.018900,2.96978,5.755687,6.602938,8.837000,9.167312,27.722601,1828.880005,4
3,21.0491,105.8831,197.524994,102181.0,78.663696,15.516100,2.77573,8.956062,10.830875,13.301438,13.640625,27.722601,1828.880005,4
4,21.0491,105.8831,287.825012,102072.0,75.866699,17.658199,3.12869,12.547000,12.891687,15.638813,12.034750,27.722601,1828.880005,4
5,21.0491,105.8831,265.541992,102019.0,71.919502,16.782400,1.43104,11.180250,10.068938,12.747250,11.967813,27.722601,1828.880005,4
6,21.0491,105.8831,358.341003,101927.0,72.536797,19.108400,3.35485,12.501250,13.821375,15.252125,12.808813,27.722601,1828.880005,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,454.903015,102068.0,43.484299,16.768499,3.66034,7.437375,5.100375,3.930875,6.508750,442.738007,4993.080078,5
5456,21.0197,105.8147,400.079010,101755.0,49.469700,19.499901,2.41066,13.452063,12.091188,5.180625,9.206437,442.738007,4993.080078,5
5457,21.0197,105.8147,326.407013,101510.0,66.391800,22.555500,1.45771,17.722375,16.551500,13.518500,18.223813,442.738007,4993.080078,5
5458,21.0197,105.8147,468.182007,101796.0,64.951401,23.062800,4.10253,16.676687,14.273500,16.221062,18.177937,442.738007,4993.080078,5


In [ ]:
X.shape

(4679, 14)

In [ ]:
# List of RMSE on all folds
# rmses_rf = []
# mapes_rf = []
# r2s_rf = []

# rmses_lr = []
# mapes_lr = []
# r2s_lr = []

# Fold tracking
i = 1
# Depth of Random Forest
depth = 100
# Number of split
split_number = 10

map_depth_rmse = {}
map_depth_r2 = {}

# How many number want to round with loss (1.424 if precision =3 for example)
precision = 5

def get_key(dict, val):
  for key, value in dict.items():
        if val == value:
            return key

  return "key doesn't exist"

# kf = KFold(n_splits=split_number, random_state=None, shuffle=None)
# kf.get_n_splits(X)

# for train_index, test_index in kf.split(X):
#   min_rmse_rf = 10000
#   min_r2_rf = 10000
#   min_mape_rf = 10000

#   min_rmse_lr = 10000
#   min_r2_lr = 10000
#   min_mape_lr = 10000

#   # print("TRAIN:", train_index, "TEST:", test_index)
#   X_train, X_test = X_arr[train_index], X_arr[test_index]
#   y_train, y_test = y_arr[train_index], y_arr[test_index]
#   # print(len(train_index), len(test_index))

#   print('Fold {}:'.format(i))

  # Random Forest
for d in range(1, depth):
  kf = KFold(n_splits=split_number, random_state=None, shuffle=True)
  kf.get_n_splits(X)

  rmses_rf = []
  mapes_rf = []
  r2s_rf = []

  rmses_lr = []
  mapes_lr = []
  r2s_lr = []

  for train_index, test_index in kf.split(X):
    min_rmse_rf = 10000
    min_r2_rf = 10000
    min_mape_rf = 10000

    min_rmse_lr = 10000
    min_r2_lr = 10000
    min_mape_lr = 10000

    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_arr[train_index], X_arr[test_index]
    y_train, y_test = y_arr[train_index], y_arr[test_index]
    # print(len(train_index), len(test_index))

    # print('Fold {}:'.format(i))
    rf = RandomForestRegressor(n_estimators = d, random_state = 20)
    rf.fit(X_train, y_train)
    rf_predicts = rf.predict(X_test)
    # Calculate RMSE
    rmse_rf = math.sqrt(mean_squared_error(y_test, rf_predicts))
    # Calculate R2 Score
    r2_rf = r2_score(y_test, rf_predicts)
    # Calculate mean absolute error (MAE NOT MAPE)
    # mape_rf = mean_absolute_error(y_test, rf_predicts)

    # With each depth, calculate and save losses of all fold
    rmses_rf.append(rmse_rf)
    r2s_rf.append(r2_rf)
    mapes_rf.append(min_mape_rf)

  # After each depth, save average loss to find best depth
  average_rmse_rf = round(np.average(rmses_rf), precision)
  average_r2_rf = round(np.average(r2s_rf), precision)

  map_depth_rmse[d] = average_rmse_rf
  map_depth_r2[d] = average_r2_rf
  
  print('Depth =', d)
  print('Average RMSE on all folds using Random Forest:', average_rmse_rf)
  print('Average R2 on all folds using Random Forest:', average_r2_rf)
  # print('Average MAPE on all folds using Random Forest:', np.average(mapes_rf))
  print()

# print(map_depth_rmse)
# print(map_depth_r2)

min_rmse = min(map_depth_rmse.values())
min_r2 = min(map_depth_r2.values())

print('Best depth for RMSE is {} with RMSE = {}'.format(get_key(map_depth_rmse, min_rmse), min_rmse))
print('Best depth for R2 is {} with R2 = {}'.format(get_key(map_depth_r2, min_r2), min_r2))

  # # Linear Regression
  # lr = LinearRegression()
  # lr.fit(X_train, y_train)
  # lr_predicts = lr.predict(X_test)
  # # Calculate RMSE
  # rmse_lr = math.sqrt(mean_squared_error(y_test, lr_predicts))
  # # Calculate R2 Score
  # r2_lr = r2_score(y_test, lr_predicts)
  # # Calculate mean absolute error (MAE NOT MAPE)
  # mape_lr = mean_absolute_error(y_test, lr_predicts)

  # if rmse_lr < min_rmse_lr:
  #   min_rmse_lr = rmse_lr
  # if r2_lr < min_r2_lr:
  #   min_r2_lr = r2_lr
  # if mape_lr < min_mape_lr:
  #   min_mape_lr = mape_lr

  # # a={'true': y_test, 'predict':predicts}
  # # display(pd.DataFrame(a))

  # rmses_lr.append(min_rmse_lr)
  # r2s_lr.append(min_r2_lr)
  # mapes_lr.append(min_mape_lr)

  # print('Min RMSE for Linear Regression:'.format(i), min_rmse_lr)
  # print('Min R2 for Linear Regression:'.format(i), min_r2_lr)
  # print('Min MAPE for Linear Regression:'.format(i), min_mape_lr)
  # print()
  # print()

  # i += 1

# print()
# print('Average RMSE on all folds using Linear Regression:', np.average(rmses_lr))
# print('Average R2 on all folds using Linear Regression:', np.average(r2s_lr))
# print('Average MAPE on all folds using Linear Regression:', np.average(mapes_lr))


Depth = 1
Average RMSE on all folds using Random Forest: 13.94362
Average R2 on all folds using Random Forest: 0.14204

Depth = 2
Average RMSE on all folds using Random Forest: 12.11647
Average R2 on all folds using Random Forest: 0.35329

Depth = 3
Average RMSE on all folds using Random Forest: 11.24943
Average R2 on all folds using Random Forest: 0.44165

Depth = 4
Average RMSE on all folds using Random Forest: 10.74682
Average R2 on all folds using Random Forest: 0.49128

Depth = 5
Average RMSE on all folds using Random Forest: 10.60709
Average R2 on all folds using Random Forest: 0.50461

Depth = 6
Average RMSE on all folds using Random Forest: 10.5632
Average R2 on all folds using Random Forest: 0.50794

Depth = 7
Average RMSE on all folds using Random Forest: 10.49598
Average R2 on all folds using Random Forest: 0.51403

Depth = 8
Average RMSE on all folds using Random Forest: 10.27498
Average R2 on all folds using Random Forest: 0.5348

Depth = 9
Average RMSE on all folds using 

In [ ]:
max_r2 = max(map_depth_r2.values())
print('Best depth for R2 is {} with R2 = {}'.format(get_key(map_depth_r2, max_r2), max_r2))

Best depth for R2 is 97 with R2 = 0.59161
